In [54]:
import random

class Sentiment:
    negative = "Negative"
    neutral = "Neutral"
    positive = "Positive"

class Review:
    def __init__(self, text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.negative
        elif self.score == 3:
            return Sentiment.neutral
        else: #Score > 3
            return Sentiment.positive
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
      
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.negative, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.positive, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        
        

### Load Data

In [33]:
import json

reviews = []
file_name = 'Books_small_10000.json'
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

### Prep Data

In [56]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)


In [73]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.positive))
print(train_y.count(Sentiment.negative))

436
436


### Bag of words vectorization

In [87]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#The machine deals data only with matrices and numbers, 
#so we convert the words into a number array by the Counntvectorizer method(bag of words).

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())


This book can only be described as a mess. The characters are not well developed, heck, seems most of them need mental health professionals. I can imagine the author was attempting to write without benefit of any research. She fails to develop a storyline that makes sense. Additionally, she fails to explain how or why things that a vital to the story happens, such as the death of one of the main characters or how another of the bad guys appears at the final scene. Such an unlikely story that at times it borders on the comedic..
[[0. 0. 0. ... 0. 0. 0.]]


## Classification

### Linear SVM

In [88]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['Positive'], dtype='<U8')

#### Decision Tree

In [89]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['Negative'], dtype='<U8')

#### Naive Bayes

In [90]:
from sklearn.naive_bayes import MultinomialNB

clf_gnb = MultinomialNB()
clf_gnb.fit(train_x_vectors, train_y)

clf_gnb.predict(test_x_vectors[0])

array(['Positive'], dtype='<U8')

#### Logistic Regression

In [91]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)

clf_log.predict(test_x_vectors[0])

array(['Positive'], dtype='<U8')

### Evaluation

In [92]:
# Mean Accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6610576923076923
0.8125
0.8052884615384616


In [93]:
# F1 Scores
from sklearn.metrics import f1_score

# Our Model shows a good score on positive but performs very poorly on Neutral and Negative so we have to improve them
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.positive, Sentiment.negative])

#f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels=[Sentiment.positive, Sentiment.neutral, Sentiment.negative])

array([0.80582524, 0.80952381])

In [94]:
test_set = ['I throughly enjoyed this, 5 stars', 'bad book do not buy', 'horrible waste of time']
# Transforming the list into a vector
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['Positive', 'Negative', 'Negative'], dtype='<U8')

### Tuning our Model (with Grid Search)

In [97]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf'), 'C': (1,4,8,16,32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(train_x_vectors, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [98]:
print(clf_svm.score(test_x_vectors, test_y))

0.8076923076923077


### Saving Model

In [100]:
import pickle

with open('./models/sentiment_classifer.pkl', 'wb') as f:
    pickle.dump(clf, f)

### Load Model

In [101]:
with open('./models/sentiment_classifer.pkl', 'rb') as f:
    load_clf = pickle.load(f)

In [104]:
print(test_x[0])

load_clf.predict(test_x_vectors[0])

Recommended by a friend as &#34;one of the best books I have read.&#34;  I would not go that far but it is a page turner and very touching story.  Delightful read.


array(['Positive'], dtype='<U8')